Ques 3) Multivariate

In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np


In [18]:

ettH1_data = pd.read_csv('ETTh1.csv')
ettH2_data = pd.read_csv('ETTh2.csv')
ettM1_data = pd.read_csv('ETTm1.csv')
ettM2_data = pd.read_csv('ETTm2.csv')


etth1_data = ettH1_data[['date', 'HUFL']].copy()
etth1_data['date'] = pd.to_datetime(etth1_data['date'])

etth2_data = ettH2_data[['date', 'HULL']].copy()
etth2_data['date'] = pd.to_datetime(etth2_data['date'])

ettm1_data = ettM1_data[['date', 'HUFL']].copy()
ettm1_data['date'] = pd.to_datetime(ettm1_data['date'])

ettm2_data = ettM2_data[['date', 'HULL']].copy()
ettm2_data['date'] = pd.to_datetime(ettm2_data['date'])


ettH1_data['date'] = etth1_data['date']
ettH2_data['date'] = etth2_data['date']
ettM1_data['date'] = ettm1_data['date']
ettM2_data['date'] = ettm2_data['date']

etth1_data = ettH1_data[['date', 'HUFL']]
etth1_data['date'] = pd.to_datetime(etth1_data['date'])
etth1_data.set_index('date', inplace=True)
etth1_values = etth1_data.values.astype(float)

etth2_data = ettH2_data[['date', 'HULL']]
etth2_data['date'] = pd.to_datetime(etth2_data['date'])
etth2_data.set_index('date', inplace=True)
etth2_values = etth2_data.values.astype(float)

ettm1_data = ettM1_data[['date', 'HUFL']]
ettm1_data['date'] = pd.to_datetime(ettm1_data['date'])
ettm1_data.set_index('date', inplace=True)
ettm1_values = ettm1_data.values.astype(float)

ettm2_data = ettM2_data[['date', 'HULL']]
ettm2_data['date'] = pd.to_datetime(ettm2_data['date'])
ettm2_data.set_index('date', inplace=True)
ettm2_values = ettm2_data.values.astype(float)


sequence_length = 50

scaler_etth1 = MinMaxScaler()
scaler_etth2 = MinMaxScaler()
scaler_ettm1 = MinMaxScaler()
scaler_ettm2 = MinMaxScaler()

etth1_scaled = scaler_etth1.fit_transform(etth1_values)
etth2_scaled = scaler_etth2.fit_transform(etth2_values)
ettm1_scaled = scaler_ettm1.fit_transform(ettm1_values)
ettm2_scaled = scaler_ettm2.fit_transform(ettm2_values)

def create_lstm_model(sequence_length, num_features):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, num_features)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_multivariate_lstm(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :])
        y.append(data[i + sequence_length, 0])

    X, y = np.array(X), np.array(y)
    model = create_lstm_model(sequence_length, data.shape[1])
    model.fit(X, y, epochs=5, batch_size=32)

    predictions = []
    current_sequence = X[-1]
    for i in range(10 * sequence_length):
        prediction = model.predict(current_sequence.reshape(1, sequence_length, data.shape[1]))[0, 0]
        predictions.append(prediction)
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = prediction

    return np.array(predictions)

predictions_ETTh1 = train_multivariate_lstm(etth1_scaled, sequence_length)
predictions_ETTh2 = train_multivariate_lstm(etth2_scaled, sequence_length)
predictions_ETTm1 = train_multivariate_lstm(ettm1_scaled, sequence_length)
predictions_ETTm2 = train_multivariate_lstm(ettm2_scaled, sequence_length)

combined_matrix = np.column_stack((predictions_ETTh1, predictions_ETTh2, predictions_ETTm1, predictions_ETTm2))
print(combined_matrix)


<ipython-input-18-fc32360b4507>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth1_data['date'] = pd.to_datetime(etth1_data['date'])
<ipython-input-18-fc32360b4507>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth2_data['date'] = pd.to_datetime(etth2_data['date'])
<ipython-input-18-fc32360b4507>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

Epoch 1/5
543/543 [==============================] - 8s 8ms/step - loss: 0.0162
Epoch 2/5
543/543 [==============================] - 5s 9ms/step - loss: 0.0044
Epoch 3/5
543/543 [==============================] - 4s 8ms/step - loss: 0.0029
Epoch 4/5
543/543 [==============================] - 4s 8ms/step - loss: 0.0028
Epoch 5/5
1/1 [==============================] - 0s 22ms/step
Epoch 1/5
543/543 [==============================] - 8s 8ms/step - loss: 0.0032
Epoch 2/5
543/543 [==============================] - 4s 7ms/step - loss: 0.0013
Epoch 3/5
543/543 [==============================] - 4s 8ms/step - loss: 0.0011
Epoch 4/5
543/543 [==============================] - 4s 8ms/step - loss: 0.0011
Epoch 5/5
1/1 [==============================] - 0s 20ms/step
Epoch 1/5
2176/2176 [==============================] - 22s 8ms/step - loss: 0.0026
Epoch 2/5
2176/2176 [==============================] - 19s 9ms/step - loss: 0.0011
Epoch 3/5
2176/2176 [==============================] - 18s 8ms/step - 

In [19]:
len_ETTh1 = len(predictions_ETTh1)
len_ETTh2 = len(predictions_ETTh2)
len_ETTm1 = len(predictions_ETTm1)
len_ETTm2 = len(predictions_ETTm2)

print(len_ETTh1, len_ETTh2, len_ETTm1, len_ETTm2)


500 500 500 500


In [20]:
import pandas as pd

date_range = pd.date_range(start='2023-01-01', periods=len_ETTh1, freq='H')

combined_matrix = pd.DataFrame({
    'Date': date_range,
    'Predictions_ETTh1': predictions_ETTh1,
    'Predictions_ETTh2': predictions_ETTh2,
    'Predictions_ETTm1': predictions_ETTm1,
    'Predictions_ETTm2': predictions_ETTm2
})

print(combined_matrix)


                   Date  Predictions_ETTh1  Predictions_ETTh2  \
0   2023-01-01 00:00:00           0.753957           0.569620   
1   2023-01-01 01:00:00           0.799140           0.576866   
2   2023-01-01 02:00:00           0.822796           0.583336   
3   2023-01-01 03:00:00           0.834829           0.589415   
4   2023-01-01 04:00:00           0.843759           0.595190   
..                  ...                ...                ...   
495 2023-01-21 15:00:00           0.833997           1.137625   
496 2023-01-21 16:00:00           0.861818           1.137625   
497 2023-01-21 17:00:00           0.880593           1.137626   
498 2023-01-21 18:00:00           0.895708           1.137626   
499 2023-01-21 19:00:00           0.910481           1.137626   

     Predictions_ETTm1  Predictions_ETTm2  
0             0.746425           0.626928  
1             0.756688           0.632859  
2             0.766392           0.637314  
3             0.775108           0.640694  